In [11]:
import pandas as pd
import numpy as np
import re
import requests
from io import StringIO
import os

In [2]:
# Hacemos request de un dataset del PIB de diferentes páises y zonas geopolíticas
url = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.CD?format=json&per_page=20000"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()[1]  # Get the data from the second element of the response
    df = pd.json_normalize(data)  # Convert the JSON data into a pandas DataFrame
    print(df.head())
else:
    print("Request failed with status code:", response.status_code)

  countryiso3code  date        value unit obs_status  decimal    indicator.id   
0             AFE  2021  1549.772730                        1  NY.GDP.PCAP.CD  \
1             AFE  2020  1363.540741                        1  NY.GDP.PCAP.CD   
2             AFE  2019  1512.270553                        1  NY.GDP.PCAP.CD   
3             AFE  2018  1564.734340                        1  NY.GDP.PCAP.CD   
4             AFE  2017  1628.586788                        1  NY.GDP.PCAP.CD   

                indicator.value country.id                country.value  
0  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
1  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
2  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
3  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
4  GDP per capita (current US$)         ZH  Africa Eastern and Southern  


In [3]:
df

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2021,1549.772730,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern
1,AFE,2020,1363.540741,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern
2,AFE,2019,1512.270553,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern
3,AFE,2018,1564.734340,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern
4,AFE,2017,1628.586788,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...,...
16487,ZWE,1964,282.376856,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe
16488,ZWE,1963,277.532515,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe
16489,ZWE,1962,275.966139,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe
16490,ZWE,1961,279.332656,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe


In [4]:
#cambiamos el nombre de columnas
df = df.rename(columns={'countryiso3code': 'country_code', 'indicator.id': 'indicator_id', 'indicator.value': 'indicator_value', 'country.id': 'country_id', 'country.value': 'country_value'})


In [5]:
df = df.drop(['unit', 'obs_status','decimal','indicator_id'], axis=1)


In [6]:
df

,country_code,date,value,indicator_value,country_id,country_value
0,AFE,2021,1549.772730,GDP per capita (current US$),ZH,Africa Eastern and Southern
1,AFE,2020,1363.540741,GDP per capita (current US$),ZH,Africa Eastern and Southern
2,AFE,2019,1512.270553,GDP per capita (current US$),ZH,Africa Eastern and Southern
3,AFE,2018,1564.734340,GDP per capita (current US$),ZH,Africa Eastern and Southern
4,AFE,2017,1628.586788,GDP per capita (current US$),ZH,Africa Eastern and Southern
...,...,...,...,...,...,...
16487,ZWE,1964,282.376856,GDP per capita (current US$),ZW,Zimbabwe
16488,ZWE,1963,277.532515,GDP per capita (current US$),ZW,Zimbabwe
16489,ZWE,1962,275.966139,GDP per capita (current US$),ZW,Zimbabwe
16490,ZWE,1961,279.332656,GDP per capita (current US$),ZW,Zimbabwe


In [7]:
format_currency = lambda x: "${:,.2f}".format(x)
df['gdp'] = df['value'].apply(format_currency)
df = df.drop('value', axis=1)


,country_code,date,indicator_value,country_id,country_value,gdp
0,AFE,2021,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,549.77"
1,AFE,2020,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,363.54"
2,AFE,2019,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,512.27"
3,AFE,2018,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,564.73"
4,AFE,2017,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,628.59"
...,...,...,...,...,...,...
16487,ZWE,1964,GDP per capita (current US$),ZW,Zimbabwe,$282.38
16488,ZWE,1963,GDP per capita (current US$),ZW,Zimbabwe,$277.53
16489,ZWE,1962,GDP per capita (current US$),ZW,Zimbabwe,$275.97
16490,ZWE,1961,GDP per capita (current US$),ZW,Zimbabwe,$279.33


In [14]:
output_path = 'ironhack/Project-2---ESS-DATA/data/cleanpib.csv'
output_fullpath = os.path.abspath(output_path)
output_dir = os.path.dirname(output_fullpath)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
df.to_csv(output_fullpath, index=False)